# 랜덤포레스트

### 개별 트리 모델의 단점
- 계층적 구조로 인해 중간에 에러가 발생하면 다음 단계로 에러가 계속 전파된다.
- 학습 데이터의 미세한 변동에도 최종 결과에 큰 영향을 미친다.
- 적은 개수의 노이즈에도 크게 영향을 받는다.
- 나무의 최종노드 개수를 늘리면 과적합 위험 (Low Bias, Large Variance)

위의 문제를 해결하기 위해 __랜덤 포레스트 (Random forest)__ 를 사용

### 랜덤 포레스트 개요
- 다수의 의사결정 나무모델에 의한 예측을 종합하는 앙상블 방법
- 일반적으로 하나의 의사결정나무 모델 보다 높은 예측 정확성을 보인다.
- 관측치 수에 비해 변수의 수가 많은 고차원 데이터에서 중요 변수 선택 비법으로 널리 활용된다.  
- 핵심 아이디어 : Diversity, Random 확보
        1. 여러개의 Training data를 생성하여 각 데이터마다 개별 의사 결정나무 모델 구축 - Bagging (Bootstrap Aggregating : 각각의 bootstrap 샘플로 부터 생성된 모델을 합침
        2. 의사결정나무 모델 구축 시 변수 무작위로 선택 - Radom subspace  
        3. 개별 tree의 정확도, 독립성이 높을수록 random forest 의 성능이 높아진다. 

   
- 변수의 중요도 
    - 랜덤 포레스트는 선형 회귀모델/로지스틱 회기모델과는 달리 개별 변수가 통계적으로 얼마나 유의한지에 대한 정보를 제공하지 않는다.
    - 대신 랜덤 포레스트는 다음과 같은 간접적인 방식으로 변수의 중요도를 결정
        - 1단계 : 원래 데이터 집합에 대해서 Out of bag(OOB) Error 을 구함
        - 2단계 : 특정 변수의 값을 임의로 뒤섞은 데이터 집합에 대해서 OOB Error를 구함
        - 3단계 : 개별 변수의 중요도는 2단계와 1단계 OOB Error 차이의 평균과 분산을 고려하여 결정, 차이가 크다면 성능이 굉장히 나쁘다는걸 알 수 있다. 
        
      
- 랜덤 포레스트의 하이퍼 파라미터 
    1. Decision Tree의 수 
        - Strong law of large numbers 을 만족시키기 위해 2000개 이상의 decision tree 필요
        
    2. Decision tree에서 노드 분할 시 무작위로 선택되는 변수의 수
        - 일반적으로 변수의 수에 따라 다음과 같이 추천
        - Classification : root(변수의 수)
        - Regression : 변수의 수 /3


----------------

### 단어 정리

----------------

* Bootstrapping ??
    - bootstrapping 은 샘플링 기법중 하나이다.
    - 각 모델은 서로 다른 학습 데이터셋을 이용
    - 각 데이터셋은 복원추출(sampling with replacement)을 통해 원래 데이터(original datasets) 의 수만큼 크기를 갖도록 샘플링
    - 각 데이터셋을 bootstrap set이라고 부른다 

    ![](img/20200306_150054.png)

* Bagging ( 위의 Bootstrap  + Aggregating ) ??
    - 위의 bootstrap dataset들을 얻었다면 이젠 이것들을 합쳐(aggregating) 하나의 답을 만들어야 한다.
    ![](img/20200306_150909.png)

    - aggregating 방법은 여러가지있다.   

* Random subspace
    - 의사 결정나무의 분기점을 탐색할 떄, 원래 변수의 수보다 적은 수의 변수를 임의로 선택하여 해당 변수들만을 고려대상으로 한다.
   ![](img/20200306_152006.png)

* OOB (Out of bag) data
    ![](img/20200306_162710.png)

In [11]:
# 위에 말했듯 랜덤 포레스트는 일반적으로 배깅 기법(또는 페이스팅)을 적용한 결정 트리의 앙상블 이다.
# 전형적으로 max_samples를 훈련 세트의 크기로 지정한다.
# BaggingClassifier에 DecisionTreeClassifier 를 넣어 만드는 대신, 결정 트리에 최적화되어 사용하기 편리한 RandomForestClassifier를 사용할 수 있다.
# 아래는 16개의 리프 노드를 갖는 500개의 트리로 이루어진 랜덤 포레스트 분류기를 여러 CPU 코어에서 훈련시키는 코드이다.

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

x, y = make_moons(n_samples = 500, noise = 0.30 , random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=42)


bag_clf = BaggingClassifier(DecisionTreeClassifier(splitter = 'random', max_leaf_nodes=16,random_state=42),
                            n_estimators= 500, max_samples=1.0, bootstrap=True, n_jobs = -1,
                            random_state=42)

bag_clf.fit(x_train, y_train)
y_pred = bag_clf.predict(x_test)


from sklearn.ensemble import RandomForestClassifier 
rnd_clf = RandomForestClassifier(n_estimators = 500, max_leaf_nodes=16, n_jobs = -1, random_state=42)
rnd_clf.fit(x_train, y_train)

y_pred_rf = rnd_clf.predict(x_test)

np.sum (y_pred == y_pred_rf)/ len(y_pred)
# 거의 동일한 예측을 보인다.

0.976

RandomForestClassifier은 몇가지 예외가 있긴 하지만, 트리의 성장을 조절하기 위한 DecisionTreeClassifier의 매개변수와 앙상블 자체를 제어하는 데 필요한 BaggingClassifier의 매개변수를 모두 가지고 있다.  

랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신, 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입합니다. 이는 결국, 트리를 더욱 다양하게 만들고, 분산을 낮추어 더 훌륭한 모델을 만들어 낸다. 